#### Send ontology file for importation

In [1]:
import requests

app_api_host = 'https://api.app.lettria.com'
file_name = 'ontology.owl'
file_location = './data/'
api_route = app_api_host + '/app/knowledge/import'
jwt_token_file = 'jwt_token.txt'
jwt_token = open(jwt_token_file, 'r').read()
knowlede_id = None

with open(file_location + file_name, 'rb') as file:
    response = requests.post(
        api_route,
        files=[('file', ( file_name, file, "application/rdf+xml"))],
        headers={"Authorization":"LettriaProKey " + jwt_token}
    )
    print(response.json())
    knowlede_id = response.json()['knowledgeId']

{'message': 'ok', 'knowledgeId': '657343a3911ed5002428cad8'}


#### Polling to wait for the end of the import

In [2]:
import time
import requests

api_route = app_api_host + '/app/knowledge/read'
importation_is_finished = False

knowledge = None
importation_status = None
while not importation_is_finished:
    response = requests.post(
        api_route,
        headers={"Authorization":"LettriaProKey " + jwt_token}
    ).json()
    imported_knowledge = next((item for item in response['documents'] if item["_id"] == knowlede_id), None)
    importation_status = imported_knowledge.get('importationStatus')
    print(importation_status + "      ", end='\r')
    importation_is_finished = importation_status != 'PROCESSING'
    time.sleep(1)
if importation_status != 'SUCCESS':
    print('Importation failed')
    exit(1)

#### Extract document text chunks

In [3]:
api_host = 'https://api.lettria.com'
api_route = api_host + '/parse'
document_file_name = 'file.txt'
document_file_location = './data/'

chunks = []
with open(document_file_location + document_file_name, 'rb') as file:
    response = requests.post(
        api_route,
        files=[('file', ( document_file_name, file, "application/text"))],
        headers={"Authorization":"LettriaProKey " + jwt_token}
    ).json()
    text_chunks = response['chunks']

text_chunks = [chunk['content'] for chunk in text_chunks if chunk['type'] == 'text']
print(text_chunks)

['In the domain of our ontology, we define various classes such as Departments, Employees, External, Human_Resource, Intern, Internal, International, Junior, Local, Manager, Marketing, Office, Operations, and Senior.', "We establish relationships using object properties like 'has,' 'have,' 'is_a_kind_of,' 'is_a_part_of,' 'sub-departments,' and 'types.' Additionally, we utilize data properties such as 'createdON,' 'depID,' 'empContact,' 'empID,' 'empName,' and 'joinedON.'", 'Individuals are instances of classes, and we have named individuals like Harry, Hina, Sina, Tara, delhi_branch, france_branch, mumbai_branch, and opcore.', 'Examples of class assertions include stating that Harry is a Senior, Hina is a Junior, Sina is a Manager, Tara is an Intern, delhi_branch is Local, france_branch is International, mumbai_branch is Human_Resource, and opcore is Operations.', "We assert data property values for individuals, like Harry's employee contact, ID, name, and joining date. Similar asserti

#### Text to graph using Lettria structuration endpoint

In [4]:
api_route = api_host + '/structuration'

response = requests.post(
    api_route,
    json={
        "documents": text_chunks,
        "options":{
            "rdf_export": True
        }
    },
    headers={"Authorization":"LettriaProKey " + jwt_token}
).json()

rdf_string = response['rdf']
print(rdf_string)

@prefix card: <https://data.lettria.com/card/> .
@prefix d: <https://doc.lettria.com/ontology/1.0/Data/> .
@prefix info: <https://data.lettria.com/info/> .
@prefix l: <https://doc.lettria.com/ontology/1.0/> .
@prefix m: <https://doc.lettria.com/ontology/1.0/Metadata/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<https://data.lettria.com/project/63f642b44aaf52003163cecc> a l:Project ;
    rdfs:label "63f642b44aaf52003163cecc" ;
    l:Document <https://data.lettria.com/document/63f642b44aaf52003163cecc_document_0>,
        <https://data.lettria.com/document/63f642b44aaf52003163cecc_document_1>,
        <https://data.lettria.com/document/63f642b44aaf52003163cecc_document_2>,
        <https://data.lettria.com/document/63f642b44aaf52003163cecc_document_3>,
        <https://data.lettria.com/document/63f642b44aaf52003163